# ACCESS Metrics report for Purdue

In [ ]:
PROVIDER = 'Purdue'
RESOURCE_RENAMES = {
    'Purdue Anvil CPU': 'Anvil CPU',
    'Purdue Anvil GPU': 'Anvil GPU',
}

In [ ]:
YEAR = 2025
QUARTER_START_DATE_SUFFIX = '-01-01'
QUARTER_START_DATE = str(YEAR) + QUARTER_START_DATE_SUFFIX
QUARTER_END_DATE_SUFFIX = '-03-31'
QUARTER_END_DATE = str(YEAR) + QUARTER_END_DATE_SUFFIX
TWO_YEARS_AGO_QUARTER_START_DATE = str(YEAR - 2) + QUARTER_START_DATE_SUFFIX

In [ ]:
# This cell will be removed once JWT implementation is complete.
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(Path(os.path.expanduser('~/xdmod-data.env')), override=True)
os.environ['XDMOD_API_TOKEN'] = os.environ['PROD_API_TOKEN']

In [ ]:
#import sys
#! {sys.executable} -m pip install --upgrade 'xdmod-data>=1.0.0,<2.0.0' python-dotenv tabulate

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import xdmod_data.themes
pio.templates.default = 'timeseries'
from xdmod_data.warehouse import DataWarehouse
from IPython.display import display, Markdown
def display_df_md_table(df):
    return display(Markdown(df.replace('\n', '<br/>', regex=True).to_markdown(floatfmt=',.0f')))
dw = DataWarehouse('https://xdmod.access-ci.org')

In [ ]:
def two_year_line_plot_by_quarter(
    y=None,
    resource=None,
    dimension=None,
    nlargest=0,
    vertical_legend=False,
):
    if y == 'projects':
        metric = 'Number of Allocations: Active'
        metric_label = 'Number of Active Projects'
    elif y == 'users':
        metric = 'Number of Users: Active'
        metric_label = 'Number of Active Users'
    elif y == 'ace':
        metric = 'ACCESS Credit Equivalents Charged: Total (SU)'
        metric_label = 'ACCESS Credit Equivalents Charged'
    if resource == 'all':
        dimension = dimension_label = 'Resource'
        filters = {
            'Service Provider': PROVIDER,
        }
    else:
        filters = {
            'Resource': resource,
        }
    if dimension == 'pfos':
        dimension = 'Parent Science'
        dimension_label = 'Parent Field of Science'
    elif dimension == 'academic status':
        dimension = 'User NSF Status'
        dimension_label = 'User Academic Status'
    elif dimension == 'project':
        dimension = 'Allocation'
        dimension_label = 'Project'
    with dw:
        df = dw.get_data(
            duration=(TWO_YEARS_AGO_QUARTER_START_DATE, QUARTER_END_DATE),
            realm='Jobs',
            metric=metric,
            dimension=dimension,
            dataset_type='timeseries',
            aggregation_unit='Quarter',
            filters=filters,
        )
    df = df.rename(
        columns={
            **RESOURCE_RENAMES,
            **{dimension: dimension_label}
        }
    )
    df = df.reset_index(names='Date')
    df = pd.melt(
        df,
        id_vars=['Date'],
        var_name=dimension_label,
        value_name=metric_label,
    )
    top_dimension_labels = None
    category_orders = None
    title = metric_label + (
        (f' on {resource}')
        if resource != 'all'
        else ''
    ) + f' by {dimension_label} by Quarter, Last Two Years'
    if nlargest > 0:
        top_dimension_labels = df[
            df['Date'] == QUARTER_START_DATE
        ].nlargest(
            nlargest,
            metric_label,
        )[dimension_label].tolist()
        df = df[df[dimension_label].isin(top_dimension_labels)]
        category_orders = {
            dimension_label: top_dimension_labels,
        }
        title += ', top {nlargest}'
    plot = px.line(
        df,
        x='Date',
        y=metric_label,
        title=title,
        color=dimension_label,
        markers=True,
        category_orders=category_orders,
    )
    plot.update_traces(
        hovertemplate='%{y:,.0f}',
    )
    plot.update_layout(
        xaxis_tickformat='Q%q %Y',
        hovermode='x unified',
        hoverlabel_namelength=-1,
    )
    if vertical_legend:
        plot.update_layout(
            legend_orientation='v',
            legend_xanchor='left',
            legend_x=0,
            legend_yanchor='bottom',
            legend_y=-1.3
        )
    plot.show()
    return top_dimension_labels

## Active projects

### Total

In [ ]:
two_year_line_plot_by_quarter(
    y='projects',
    resource='all',
)

### By Parent Field of Science

In [ ]:
two_year_line_plot_by_quarter(
    y='projects',
    resource='Purdue Anvil CPU',
    dimension='pfos',
)

In [ ]:
two_year_line_plot_by_quarter(
    y='projects',
    resource='Purdue Anvil GPU',
    dimension='pfos',
)

In [ ]:
top_projects = two_year_line_plot_by_quarter(
    y='ace',
    resource='Purdue Anvil CPU',
    dimension='project',
    nlargest=10,
    vertical_legend=True,
)

In [ ]:
dimensions = ['PI', 'Parent Science']
aces = []
dimension_counts = {}
with dw:
    for dimension in dimensions:
        dimension_counts[dimension] = []
        for project in top_projects:
            df = dw.get_data(
                duration=(QUARTER_START_DATE, QUARTER_END_DATE),
                realm='Jobs',
                metric='ACCESS Credit Equivalents Charged: Total (SU)',
                dimension=dimension,
                dataset_type='aggregate',
                aggregation_unit='quarter',
                filters={
                    'Allocation': project,
                    'Resource': 'Purdue Anvil CPU',
                },
            )
            dimension_counts[dimension].append(df.index[0])
            if dimension == 'PI':
                aces.append(df.iloc[0])

In [ ]:
data = [
    top_projects,
    aces,
]
for dimension_values in dimension_counts.values():
    data.append(dimension_values)
df = pd.DataFrame(data).transpose()
df.columns = ['Project', 'ACEs Charged'] + ['PI', 'Parent Field of Science']
display_df_md_table(df)

## Active users

In [ ]:
two_year_line_plot_by_quarter(
    y='users',
    resource='all',
    dimension=None,
)

In [ ]:
two_year_line_plot_by_quarter(
    y='users',
    resource='Purdue Anvil CPU',
    dimension='pfos',
)

In [ ]:
two_year_line_plot_by_quarter(
    y='users',
    resource='Purdue Anvil GPU',
    dimension='pfos',
)

In [ ]:
two_year_line_plot_by_quarter(
    y='users',
    resource='Purdue Anvil CPU',
    dimension='academic status',
)

In [ ]:
two_year_line_plot_by_quarter(
    y='users',
    resource='Purdue Anvil GPU',
    dimension='academic status',
)